In [ ]:
import json
from pprint import pprint
import re
from os import listdir, SEEK_END
import datetime
import random

In [ ]:
list_re = [
    r"(\/sarcasm)",
    r"(\[[!?]\](?!\())",
    r"(:\^(?! ?[D)(\[\]]))",
    r"(<\/?sarcasm>)",
    r"(&lt;\/?sarcasm&gt)",
    r"(#sarcasm)",
    r"( \/s(?![a-zA-Z0-9]))"
          ]
sarc = re.compile(r"(\.~)|(\. ~)")
strike = re.compile(r"(?P<strike>~~.*~~)")
sarcasm_re = re.compile('|'.join(list_re))

filename_re = re.compile(r"""(RC_20\d{2}-\d{2}(?!\.bz2))""")

In [ ]:
def commentGenerator():
    for name in listdir():
        if filename_re.match(name) is not None:
            start = datetime.datetime.now()
            print("Processing file %s, starttime: %s"%(name, start))
            pop = []
            for n, line in enumerate(open(name)):
            ## HERE to
            ## ___________________________________________________________________________________________
            ##
                try:
                    body = json.loads(line)['body']
                    if 10 <= len(body) <= 300:
                        compareText = body.lower()
                        if(sarcasm_re.search(compareText) is not None) or \
                        ((sarc.search(compareText) is not None) and (strike.search(compareText) is None)):
                            yield (True, body)
                        else:
                            pop.append(body)
                            if len(pop) == 1000:
                                yield (False, random.choice(pop))
                                pop = []
                except:
                    print("Error on line %s, : %s"%(n, line))
            ## 
            ## _____________________________________________________________________________________________
            ## HERE
            ## The documents are in the correct format for spark to process with it's json library
            ## There is at least 1 entry which is invalid so loading the body will result in an error
            print("Finished: " + str(datetime.datetime.now()-start))

In [ ]:
df_rdd = sqlContext.read.format('json').load('json/serious100000.json')
rdd = df_rdd.rdd
rdd.take(5)

In [ ]:
def filterComments(generator):
    import nlp
    import ml
    pop = []
    for comment in generator:
            text = comment['text'].lower()
            if 10 <= len(text) <= 300:
                if sarcasm_re.search(text) is not None or \
                (sarc.search(text) is not None and (strike.search(text) is not None)):
                    yield (True, flattenDict(feature(comment['text'], nlp.ckeanTokensReddit)))
                else:
                    pop.append(comment['text'])
                    if len(pop) == 1300:
                        yield (False, flattenDict(feature(random.choice(pop), nlp.ckeanTokensReddit)))
                        pop = []
                        
features = rdd.mapPartitions(filterComments)

def getVocab(gen):
    for sarc, features in gen:
        for key in feature:
            yield key

from dvs import DictVectorizerPartial

vocab = dict(features.mapPartitions(getVocab).unique().zipWithIndex().collect())

dvp = DictVectorizerPartial(vocab=vocab) #this is not the correct syntax

def vectorize(gen, dv):
    blocksize = 100000
    sarclst = []
    featlst = []
    for sarc, features in gen:
        sarclst.append(sarc)
        featlst.append(features)
        if len(sarclst) == 100000:
            yield (sarclst, dv.transform(featlst))
            sarclst = []
            featlst = []
    yield (sarclst, dv.transform(featlst))

vdvp = lambda gen: vectorize(gen, dvp)
csrs = features.mapPartitions(vdvp)

def vstack(a,b):
    from scipy.sparse import csr_matrix, vstack
    asarc = a[0]
    acsr = a[1]
    bsarc = b[0]
    bcsr = b[1]
    return asarc.append(bsarc), vstack([acsr,bcsr], format='csr')

(y, X) = zip*(csrs.reduce(vstack).collect)

from sklearn.linear_model import LogisticRegression
import ml

results = ml.trainTest(X,
                       y,
                       classifiers=[LogisticRegression(n_jobs=-1)],
                       reduce=0,
                       splits=4,
                       trainsize=0.8,
                       testsize=0.2)
pickle.dump(results, open('trained-logistic-classifier.pickle', 'wb'))    

In [ ]:
sarcasticCount = 0
seriousCount = 0
sarcastic = open('json/sarcastic0.json', 'a')
serious = open('json/serious0.json', 'a')
sarcastic.write('[\n')
serious.write('[\n')
for n, (sarcasm, comment) in enumerate(commentGenerator()):
    try:
        if sarcasm:
            json.dump({'text':comment}, sarcastic)
            sarcasticCount += 1
            if sarcasticCount % 100000 == 0:
                serious.write('\n]')
                sarcastic.close()
                sarcastic = open('json/sarcastic%d.json'%sarcasticCount, 'a')
            else:
                sarcastic.write(',\n')
        else:
            json.dump({'text':comment}, serious)
            seriousCount += 1
            if seriousCount % 100000 == 0:
                serious.write('\n]')
                serious.close()
                serious = open('json/serious%d.json'%seriousCount, 'a')
            else:
                serious.write(',\n')
    except:
        print("Error on index %s, : (%s, %s)"%(n, sarcasm, comment))
        sarcastic.close()
        serious.close()
        
sarcastic.close()
serious.close()

sarcasticname = sarcastic.name
seriousname = serious.name

sarcastic.close()
serious.close()

sarcastic = open(sarcasticname, 'r+b')
serious = open(seriousname, 'r+b')
sarcastic.seek(-3, SEEK_END)
sarcastic.write(bytes("\r\n]", 'utf8'))
serious.seek(-3, SEEK_END)
serious.write(bytes("\r\n]", 'utf8'))
sarcastic.close()
serious.close()

In [ ]:
sarcastic.close()
serious.close()